In [46]:
import random
import numpy as np

In [47]:
class Network(object):
    def __init__(self, sizes):
        self.num_layers = len(sizes)
        self.sizes = sizes
        self.biases = [np.random.randn(y, 1) for y in sizes[1:]]
        self.weights = [np.random.randn(y, x) for x, y in zip(sizes[:-1], sizes[1:])]
        
    def feedforward(self, a):
        """
            Return the output of the network.
            a is input
        """
        for b, w in zip(self.biases, self.weights):
            a = sigmoid(np.dot(w, a) + b)
        return a
    
    def sgd(self, training_data, epochs, mini_batch_size, eta, test_data=None):
        if test_data:
            n_test = len(test_data)
        n = len(training_data)
        
        for j in range(epochs):
            random.shuffle(training_data)
            mini_batches = [
                training_data[k:k+mini_batch_size] for k in range(0, n, mini_batch_size)
            ]
            
            for mini_batch in mini_batches:
                self.update_mini_batch(mini_batch, eta)
            
            if test_data:
                print("Epoch {0}: {1}".format(j, self.evaluate(test_data) / n_test))
            else:
                print("Epoch {0} complete".format(j))
                
    def update_mini_batch(self, mini_batch, eta):
        nabla_b = [np.zeros(b.shape) for b in self.biases]
        nabla_w = [np.zeros(w.shape) for w in self.weights]
        
        for x, y in mini_batch:
            delta_nabla_b, delta_nabla_w = self.backprop(x, y)
            nabla_b = [nb+dnb for nb, dnb in zip(nabla_b, delta_nabla_b)]
            nabla_w = [nw+dnw for nw, dnw in zip(nabla_w, delta_nabla_w)]
        
        self.weights = [w - (eta / len(mini_batch)) * nw for w, nw in zip(self.weights, nabla_w)]
        self.biases = [b - (eta / len(mini_batch)) * nb for b, nb in zip(self.biases, nabla_b)]
    
    def backprop(self, x, y):
        nabla_b = [np.zeros(b.shape) for b in self.biases]
        nabla_w = [np.zeros(w.shape) for w in self.weights]
        
        activation = x
        activations = [x]
        zs = []
        
        for b, w in zip(self.biases, self.weights):
            z = np.dot(w, activation) + b
            zs.append(z)
            activation = sigmoid(z)
            activations.append(activation)
        
        delta = self.cost_derivative(activations[-1], y) * sigmoid_prime(zs[-1])
        nabla_b[-1] = delta
        nabla_w[-1] = np.dot(delta, activations[-2].transpose())
        
        for l in range(2, self.num_layers):
            z = zs[-l]
            sp = sigmoid_prime(z)
            delta = np.dot(self.weights[-l+1].transpose(), delta) * sp
            nabla_b[-l] = delta
            nabla_w[-l] = np.dot(delta, activations[-l-1].transpose())
        return (nabla_b, nabla_w)
    
    def evaluate(self, test_data):
        test_results = [(np.argmax(self.feedforward(x)), y) for (x, y) in test_data]
        return sum(int(x == y) for (x, y) in test_results)
    
    def cost_derivative(self, output_activations, y):
        return (output_activations - y)

In [48]:
def sigmoid(z):
    return 1.0 / (1.0 + np.exp(-z))

def sigmoid_prime(z):
    return sigmoid(z) * (1 - sigmoid(z))

In [49]:
import mnist_loader

In [50]:
training_data, validation_data, test_data = mnist_loader.load_data_wrapper()

In [51]:
training_data, validation_data, test_data = list(training_data), list(validation_data), list(test_data)

In [52]:
net = Network([784, 30, 10])

In [53]:
net.sgd(training_data, 30, 10, 3.0, test_data=test_data)

Epoch 0: 0.9089
Epoch 1: 0.9226
Epoch 2: 0.9302
Epoch 3: 0.934
Epoch 4: 0.9367
Epoch 5: 0.9395
Epoch 6: 0.9413
Epoch 7: 0.9428
Epoch 8: 0.9468
Epoch 9: 0.9433
Epoch 10: 0.9426
Epoch 11: 0.9467
Epoch 12: 0.9465
Epoch 13: 0.9494
Epoch 14: 0.9467
Epoch 15: 0.9485
Epoch 16: 0.9484
Epoch 17: 0.95
Epoch 18: 0.9472
Epoch 19: 0.9502
Epoch 20: 0.9476
Epoch 21: 0.9489
Epoch 22: 0.9486
Epoch 23: 0.9487
Epoch 24: 0.9494
Epoch 25: 0.9482
Epoch 26: 0.9489
Epoch 27: 0.9479
Epoch 28: 0.9502
Epoch 29: 0.9503
